In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/processed/bike_sharing_cleaned.csv')
df

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,mixed_type_col
0,1.0,2011-01-01,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0000,3.0,13.0,16.0,702.0
1,2.0,2011-01-01,1.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0000,8.0,32.0,40.0,831.0
2,3.0,2011-01-01,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0000,5.0,27.0,32.0,175.0
3,4.0,2011-01-01,1.0,0.0,1.0,3.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0000,3.0,10.0,13.0,581.0
4,5.0,2011-01-01,1.0,0.0,1.0,4.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0000,0.0,1.0,1.0,659.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17721,13401.0,2012-07-17,3.0,1.0,7.0,13.0,0.0,2.0,1.0,1.0,0.92,0.8182,0.31,0.1940,53.0,168.0,221.0,NaN
17722,17216.0,2012-12-25,1.0,1.0,12.0,4.0,1.0,2.0,0.0,2.0,0.24,0.2576,0.87,0.0896,0.0,1.0,1.0,27.0
17723,9341.0,2012-01-30,1.0,1.0,1.0,2.0,0.0,1.0,1.0,1.0,0.24,0.2121,0.48,0.3582,1.0,6.0,7.0,NaN
17724,6268.0,2011-09-23,4.0,0.0,9.0,17.0,0.0,5.0,1.0,2.0,0.60,0.5000,1.00,0.0000,13.0,86.0,NaN,602.0


In [3]:
# Delete mixed_type_col
df =  df.drop('mixed_type_col', axis=1)

In [4]:
# Check NaN values in casual, registered, and cnt columns
casual_nan = df['casual'].isna().sum()
registered_nan = df['registered'].isna().sum()
cnt_nan = df['cnt'].isna().sum()

total_rows = len(df)

print(f"NaN values in 'casual': {casual_nan} ({casual_nan/total_rows*100:.2f}%)")
print(f"NaN values in 'registered': {registered_nan} ({registered_nan/total_rows*100:.2f}%)")
print(f"NaN values in 'cnt': {cnt_nan} ({cnt_nan/total_rows*100:.2f}%)")

# Sum all NaN values from these 3 columns
total_nan = casual_nan + registered_nan + cnt_nan
print(f"\nTotal NaN values in casual, registered, and cnt: {total_nan} ({total_nan/total_rows*100:.2f}%)")

NaN values in 'casual': 272 (1.53%)
NaN values in 'registered': 256 (1.44%)
NaN values in 'cnt': 247 (1.39%)

Total NaN values in casual, registered, and cnt: 775 (4.37%)


In [5]:
# Calculate missing values using the relationship: casual + registered = cnt

# Fill missing cnt values where casual and registered are available
df['cnt'] = df['cnt'].fillna(df['casual'] + df['registered'])

# Fill missing casual values where registered and cnt are available
df['casual'] = df['casual'].fillna(df['cnt'] - df['registered'])

# Fill missing registered values where casual and cnt are available
df['registered'] = df['registered'].fillna(df['cnt'] - df['casual'])

# Check NaN values after filling
casual_nan_after = df['casual'].isna().sum()
registered_nan_after = df['registered'].isna().sum()
cnt_nan_after = df['cnt'].isna().sum()

print("After filling missing values:")
print(f"NaN values in 'casual': {casual_nan_after} ({casual_nan_after/total_rows*100:.2f}%)")
print(f"NaN values in 'registered': {registered_nan_after} ({registered_nan_after/total_rows*100:.2f}%)")
print(f"NaN values in 'cnt': {cnt_nan_after} ({cnt_nan_after/total_rows*100:.2f}%)")

total_nan_after = casual_nan_after + registered_nan_after + cnt_nan_after
print(f"\nTotal NaN values after filling: {total_nan_after} ({total_nan_after/total_rows*100:.2f}%)")

After filling missing values:
NaN values in 'casual': 12 (0.07%)
NaN values in 'registered': 10 (0.06%)
NaN values in 'cnt': 8 (0.05%)

Total NaN values after filling: 30 (0.17%)


In [6]:
# Delete rows with NaN values in casual, registered, or cnt columns
rows_before = len(df)
df = df.dropna(subset=['casual', 'registered', 'cnt'])
rows_after = len(df)

rows_deleted = rows_before - rows_after
print(f"Rows before deletion: {rows_before}")
print(f"Rows after deletion: {rows_after}")
print(f"Rows deleted: {rows_deleted} ({rows_deleted/rows_before*100:.2f}%)")

Rows before deletion: 17726
Rows after deletion: 17711
Rows deleted: 15 (0.08%)


In [7]:
# Check pattern of negative values
print("="*80)
print("ROUNDING VALUES TO INTEGERS")
print("="*80)
print("All values in casual, registered, and cnt have been rounded to integers\n")

df['casual'] = df['casual'].round().astype(int)
df['registered'] = df['registered'].round().astype(int)
df['cnt'] = df['cnt'].round().astype(int)



print("="*80)
print("ANALYZING NEGATIVE VALUES PATTERN")
print("="*80)

negative_mask = (df['casual'] < 0) | (df['registered'] < 0) | (df['cnt'] < 0)
negative_rows = df[negative_mask].copy()

if len(negative_rows) > 0:
    # Count how many negative values in each row
    negative_rows['neg_count'] = (
        (negative_rows['casual'] < 0).astype(int) + 
        (negative_rows['registered'] < 0).astype(int) + 
        (negative_rows['cnt'] < 0).astype(int)
    )
    
    rows_with_1_neg = (negative_rows['neg_count'] == 1).sum()
    rows_with_2_neg = (negative_rows['neg_count'] == 2).sum()
    rows_with_3_neg = (negative_rows['neg_count'] == 3).sum()
    
    print(f"Total rows with negative values: {len(negative_rows)}")
    print(f"\nBreakdown:")
    print(f"  Rows with 1 negative value: {rows_with_1_neg}")
    print(f"  Rows with 2 negative values: {rows_with_2_neg}")
    print(f"  Rows with 3 negative values: {rows_with_3_neg}")
    
    print("\n" + "="*80)
    print("ROWS WITH NEGATIVE VALUES")
    print("="*80)
    display(negative_rows[['instant', 'dteday', 'casual', 'registered', 'cnt', 'neg_count']].sort_values('neg_count', ascending=False))
else:
    print("No negative values found!")

ROUNDING VALUES TO INTEGERS
All values in casual, registered, and cnt have been rounded to integers

ANALYZING NEGATIVE VALUES PATTERN
Total rows with negative values: 10

Breakdown:
  Rows with 1 negative value: 10
  Rows with 2 negative values: 0
  Rows with 3 negative values: 0

ROWS WITH NEGATIVE VALUES


,instant,dteday,casual,registered,cnt,neg_count
839,840.0,2011-02-07,-889,944,55,1
920,921.0,2011-02-10,-6508,6580,72,1
1372,1373.0,2011-03-02,488,-415,73,1
2028,2029.0,2011-03-30,942,-788,154,1
3675,3676.0,2011-06-07,66,-62,4,1
6020,6517.0,2011-09-13,3402,-3267,135,1
9773,9774.0,2012-02-17,-270,271,1,1
10852,10853.0,2012-04-02,3384,-2795,589,1
10985,10986.0,2012-04-07,6000,-5766,234,1
12010,12011.0,2012-05-20,-405,907,502,1


In [8]:
# Recalculate rows with negative values (since all have only 1 negative per row)
print("="*80)
print("RECALCULATING NEGATIVE VALUES")
print("="*80)

# For rows with negative casual: recalculate casual = cnt - registered
negative_casual_mask = df['casual'] < 0
if negative_casual_mask.sum() > 0:
    print(f"Recalculating {negative_casual_mask.sum()} rows with negative casual values...")
    df.loc[negative_casual_mask, 'casual'] = df.loc[negative_casual_mask, 'cnt'] - df.loc[negative_casual_mask, 'registered']

# For rows with negative registered: recalculate registered = cnt - casual
negative_registered_mask = df['registered'] < 0
if negative_registered_mask.sum() > 0:
    print(f"Recalculating {negative_registered_mask.sum()} rows with negative registered values...")
    df.loc[negative_registered_mask, 'registered'] = df.loc[negative_registered_mask, 'cnt'] - df.loc[negative_registered_mask, 'casual']

# For rows with negative cnt: recalculate cnt = casual + registered
negative_cnt_mask = df['cnt'] < 0
if negative_cnt_mask.sum() > 0:
    print(f"Recalculating {negative_cnt_mask.sum()} rows with negative cnt values...")
    df.loc[negative_cnt_mask, 'cnt'] = df.loc[negative_cnt_mask, 'casual'] + df.loc[negative_cnt_mask, 'registered']

# Verify no negative values remain
print("\nVerification after recalculation:")
print(f"  Negative casual: {(df['casual'] < 0).sum()}")
print(f"  Negative registered: {(df['registered'] < 0).sum()}")
print(f"  Negative cnt: {(df['cnt'] < 0).sum()}")

if (df['casual'] < 0).sum() == 0 and (df['registered'] < 0).sum() == 0 and (df['cnt'] < 0).sum() == 0:
    print("\n All negative values successfully recalculated!")
else:
    print("\n Warning: Some negative values remain!")

RECALCULATING NEGATIVE VALUES
Recalculating 4 rows with negative casual values...
Recalculating 6 rows with negative registered values...

Verification after recalculation:
  Negative casual: 4
  Negative registered: 6
  Negative cnt: 0



In [9]:
# Delete rows with remaining negative values
print("="*80)
print("DELETING ROWS WITH NEGATIVE VALUES")
print("="*80)

rows_before = len(df)

# Create mask for rows with any negative value
negative_mask = (df['casual'] < 0) | (df['registered'] < 0) | (df['cnt'] < 0)
rows_with_negatives = negative_mask.sum()

# Drop rows with negative values
df = df[~negative_mask]

rows_after = len(df)
rows_deleted = rows_before - rows_after

print(f"Rows before deletion: {rows_before}")
print(f"Rows after deletion: {rows_after}")
print(f"Rows deleted: {rows_deleted} ({rows_deleted/rows_before*100:.2f}%)")

# Verify no negative values remain
print("\nVerification:")
print(f"  Negative casual: {(df['casual'] < 0).sum()}")
print(f"  Negative registered: {(df['registered'] < 0).sum()}")
print(f"  Negative cnt: {(df['cnt'] < 0).sum()}")
print("\n All rows with negative values have been deleted!")

DELETING ROWS WITH NEGATIVE VALUES
Rows before deletion: 17711
Rows after deletion: 17701
Rows deleted: 10 (0.06%)

Verification:
  Negative casual: 0
  Negative registered: 0
  Negative cnt: 0

 All rows with negative values have been deleted!


In [10]:
# Recalculate rows where casual + registered != cnt
print("="*80)
print("FIXING INCONSISTENT ROWS (casual + registered != cnt)")
print("="*80)

# Find inconsistent rows (using integer comparison, no tolerance needed)
inconsistent_mask = (df['casual'] + df['registered']) != df['cnt']
inconsistent_count = inconsistent_mask.sum()

print(f"Rows where casual + registered != cnt: {inconsistent_count} ({inconsistent_count/len(df)*100:.2f}%)")

if inconsistent_count > 0:
    print(f"\nRecalculating cnt = casual + registered for {inconsistent_count} rows...")
    
    # Recalculate cnt as the sum of casual and registered
    df.loc[inconsistent_mask, 'cnt'] = df.loc[inconsistent_mask, 'casual'] + df.loc[inconsistent_mask, 'registered']
    
    # Verify all rows are now consistent
    still_inconsistent = ((df['casual'] + df['registered']) != df['cnt']).sum()
    
    print("\nVerification after recalculation:")
    print(f"  Inconsistent rows remaining: {still_inconsistent}")
    
    if still_inconsistent == 0:
        print("\n All rows now follow the relationship: casual + registered = cnt")
    else:
        print("\n Warning: Some inconsistent rows remain!")
else:
    print("\n All rows already follow the relationship: casual + registered = cnt")

FIXING INCONSISTENT ROWS (casual + registered != cnt)
Rows where casual + registered != cnt: 477 (2.69%)

Recalculating cnt = casual + registered for 477 rows...

Verification after recalculation:
  Inconsistent rows remaining: 0

 All rows now follow the relationship: casual + registered = cnt


In [11]:
# Function to detect outliers using IQR method
def detect_outliers(df, column_name, iqr_multiplier=1.5):
    """
    Detect outliers in a column using the IQR method.
    Returns the index of outlier rows.
    
    Parameters:
    - df: DataFrame
    - column_name: Column to analyze
    - iqr_multiplier: Multiplier for IQR (default=1.5)
    """
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - iqr_multiplier * IQR
    upper_bound = Q3 + iqr_multiplier * IQR
    
    outlier_mask = (df[column_name] < lower_bound) | (df[column_name] > upper_bound)
    outlier_indices = df[outlier_mask].index
    
    print(f"\nColumn: {column_name} (IQR multiplier: {iqr_multiplier})")
    print(f"  Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
    print(f"  Lower bound: {lower_bound:.2f}, Upper bound: {upper_bound:.2f}")
    print(f"  Number of outliers: {len(outlier_indices)} ({len(outlier_indices)/len(df)*100:.2f}%)")
    
    return outlier_indices


IQR_MULTIPLIER = 2.5

print("="*80)
print(f"OUTLIER DETECTION (IQR Multiplier: {IQR_MULTIPLIER})")
print("="*80)

casual_outliers = detect_outliers(df, 'casual', IQR_MULTIPLIER)
registered_outliers = detect_outliers(df, 'registered', IQR_MULTIPLIER)
cnt_outliers = detect_outliers(df, 'cnt', IQR_MULTIPLIER)

OUTLIER DETECTION (IQR Multiplier: 2.5)

Column: casual (IQR multiplier: 2.5)
  Q1: 4.00, Q3: 49.00, IQR: 45.00
  Lower bound: -108.50, Upper bound: 161.50
  Number of outliers: 808 (4.56%)

Column: registered (IQR multiplier: 2.5)
  Q1: 35.00, Q3: 224.00, IQR: 189.00
  Lower bound: -437.50, Upper bound: 696.50
  Number of outliers: 290 (1.64%)

Column: cnt (IQR multiplier: 2.5)
  Q1: 41.00, Q3: 289.00, IQR: 248.00
  Lower bound: -579.00, Upper bound: 909.00
  Number of outliers: 166 (0.94%)


## Outlier Detection Strategy

I am using an IQR multiplier of **2.5** instead of the standard 1.5 to detect more extreme outliers only. 

**Rationale:**
- Outliers in bike sharing data could be legitimate due to holidays, special events, or favorable weather conditions
- Using a higher threshold (2.5) helps avoid flagging legitimate high-demand periods as outliers
- This approach focuses on identifying truly anomalous data points that are likely data entry errors rather than natural variations in demand

In [12]:
# Identify rows where outliers appear in only one column
casual_only = set(casual_outliers) - set(registered_outliers) - set(cnt_outliers)
registered_only = set(registered_outliers) - set(casual_outliers) - set(cnt_outliers)
cnt_only = set(cnt_outliers) - set(casual_outliers) - set(registered_outliers)

print("\n" + "="*80)
print("OUTLIERS IN SINGLE COLUMNS")
print("="*80)
print(f"Outliers only in 'casual': {len(casual_only)}")
print(f"Outliers only in 'registered': {len(registered_only)}")
print(f"Outliers only in 'cnt': {len(cnt_only)}")

# Recalculate values for rows with outliers in only one column
print("\n" + "="*80)
print("RECALCULATING VALUES FOR SINGLE-COLUMN OUTLIERS")
print("="*80)

# For casual outliers only: recalculate casual = cnt - registered
for idx in casual_only:
    old_casual = df.loc[idx, 'casual']
    df.loc[idx, 'casual'] = df.loc[idx, 'cnt'] - df.loc[idx, 'registered']
    new_casual = df.loc[idx, 'casual']
    if abs(old_casual - new_casual) > 0.01:
        print(f"Row {idx}: casual changed from {old_casual:.2f} to {new_casual:.2f}")

# For registered outliers only: recalculate registered = cnt - casual
for idx in registered_only:
    old_registered = df.loc[idx, 'registered']
    df.loc[idx, 'registered'] = df.loc[idx, 'cnt'] - df.loc[idx, 'casual']
    new_registered = df.loc[idx, 'registered']
    if abs(old_registered - new_registered) > 0.01:
        print(f"Row {idx}: registered changed from {old_registered:.2f} to {new_registered:.2f}")

# For cnt outliers only: recalculate cnt = casual + registered
for idx in cnt_only:
    old_cnt = df.loc[idx, 'cnt']
    df.loc[idx, 'cnt'] = df.loc[idx, 'casual'] + df.loc[idx, 'registered']
    new_cnt = df.loc[idx, 'cnt']
    if abs(old_cnt - new_cnt) > 0.01:
        print(f"Row {idx}: cnt changed from {old_cnt:.2f} to {new_cnt:.2f}")

print(f"\nTotal values recalculated: {len(casual_only) + len(registered_only) + len(cnt_only)}")


OUTLIERS IN SINGLE COLUMNS
Outliers only in 'casual': 732
Outliers only in 'registered': 189
Outliers only in 'cnt': 0

RECALCULATING VALUES FOR SINGLE-COLUMN OUTLIERS

Total values recalculated: 921


In [13]:
# Find rows with outliers in 2 or more columns
all_indices = set(casual_outliers) | set(registered_outliers) | set(cnt_outliers)

multi_outlier_rows = []
for idx in all_indices:
    outlier_count = 0
    outlier_cols = []
    
    if idx in casual_outliers:
        outlier_count += 1
        outlier_cols.append('casual')
    if idx in registered_outliers:
        outlier_count += 1
        outlier_cols.append('registered')
    if idx in cnt_outliers:
        outlier_count += 1
        outlier_cols.append('cnt')
    
    if outlier_count >= 2:
        multi_outlier_rows.append((idx, outlier_count, outlier_cols))

print("="*80)
print("ROWS WITH OUTLIERS IN 2 OR MORE COLUMNS")
print("="*80)

if len(multi_outlier_rows) > 0:
    # Sort by number of outlier columns (descending)
    multi_outlier_rows.sort(key=lambda x: x[1], reverse=True)
    
    outliers_in_3 = sum(1 for _, count, _ in multi_outlier_rows if count == 3)
    outliers_in_2 = sum(1 for _, count, _ in multi_outlier_rows if count == 2)
    total_multi_outliers = outliers_in_3 + outliers_in_2
    
    print(f"Total rows with outliers in 2 or more columns: {total_multi_outliers} ({total_multi_outliers/len(df)*100:.2f}%)")
    print("\nBreakdown:")
    print(f"  Outliers in all 3 columns: {outliers_in_3} ({outliers_in_3/len(df)*100:.2f}%)")
    print(f"  Outliers in exactly 2 columns: {outliers_in_2} ({outliers_in_2/len(df)*100:.2f}%)")
    
    # Show the actual rows
    print("\n" + "="*80)
    print("DETAILED VIEW OF ROWS WITH MULTIPLE OUTLIERS")
    print("="*80)
    
    multi_outlier_indices = [idx for idx, _, _ in multi_outlier_rows]
    result_df = df.loc[multi_outlier_indices].copy()
    result_df['outlier_columns'] = [', '.join(cols) for _, _, cols in multi_outlier_rows]
    result_df['outlier_count'] = [count for _, count, _ in multi_outlier_rows]
    
    # Display relevant columns
    display_cols = ['instant', 'dteday', 'casual', 'registered', 'cnt', 'outlier_count', 'outlier_columns']
    display(result_df[display_cols].head())
else:
    print("No rows found with outliers in 2 or more columns.")

ROWS WITH OUTLIERS IN 2 OR MORE COLUMNS
Total rows with outliers in 2 or more columns: 167 (0.94%)

Breakdown:
  Outliers in all 3 columns: 9 (0.05%)
  Outliers in exactly 2 columns: 158 (0.89%)

DETAILED VIEW OF ROWS WITH MULTIPLE OUTLIERS


,instant,dteday,casual,registered,cnt,outlier_count,outlier_columns
12323,12324.0,2012-06-02,275,842,1117,3,"casual, registered, cnt"
12487,12488.0,2012-06-09,196,1018,1214,3,"casual, registered, cnt"
14169,14170.0,2012-08-18,242,6052,6294,3,"casual, registered, cnt"
14460,14461.0,2012-08-30,3906,751,4657,3,"casual, registered, cnt"
14748,14749.0,2012-09-11,168,802,970,3,"casual, registered, cnt"


In [14]:
# Delete rows with outliers in 2 or more columns
rows_before_delete = len(df)

# Get indices of rows with multiple outliers
multi_outlier_indices = [idx for idx, _, _ in multi_outlier_rows]

# Drop these rows from the dataframe
df = df.drop(index=multi_outlier_indices)

rows_after_delete = len(df)
rows_deleted_outliers = rows_before_delete - rows_after_delete

print("="*80)
print("DELETING ROWS WITH OUTLIERS IN 2 OR MORE COLUMNS")
print("="*80)
print(f"Rows before deletion: {rows_before_delete}")
print(f"Rows after deletion: {rows_after_delete}")
print(f"Rows deleted: {rows_deleted_outliers} ({rows_deleted_outliers/rows_before_delete*100:.2f}%)")
print(f"\nRemaining rows: {rows_after_delete}")

DELETING ROWS WITH OUTLIERS IN 2 OR MORE COLUMNS
Rows before deletion: 17701
Rows after deletion: 17534
Rows deleted: 167 (0.94%)

Remaining rows: 17534


In [15]:
# df.to_csv("../data/processed/bike_sharing_cleaned.csv", index=False)
# print("Data saved")